In [1]:
import models.models_one_task as md_one
import models.models_multi_task as md_multi
import datasets.iemocap as ds
from constants import *
from torchsummary import summary
import torch
import torch.nn as nn
import skorch
from skorch.helper import predefined_split
from skorch.dataset import Dataset
from skorch.classifier import NeuralNetClassifier
import skorch.callbacks as callbacks
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

In [2]:
iemocap_224_noprep_train = ds.IemocapDataset(  # Без препроцессинга, тренировочный
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER, 
    base_name='IEMOCAP-4', label_type='four', mode='train', preprocessing=False, 
    augmentation=False, padding='repeat', spectrogram_shape=224, spectrogram_type='melspec', tasks=['emotion'] 
)
iemocap_224_noprep_test = ds.IemocapDataset(  # Без препроцессинга, тестовый
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER, 
    base_name='IEMOCAP-4', label_type='four', mode='test', preprocessing=False, 
    augmentation=False, padding='repeat', spectrogram_shape=224, spectrogram_type='melspec', tasks=['emotion'] 
)

============= INITIALIZING DATASET IEMOCAP-4_four_prep-false_224_train ===============
=========================== SUCCESS! ====================================
============= INITIALIZING DATASET IEMOCAP-4_four_prep-false_224_test ===============
=========================== SUCCESS! ====================================


In [3]:
train_ds = iemocap_224_noprep_train
test_ds = iemocap_224_noprep_test
print(train_ds[0])

(tensor([[[-2.2222, -2.2222, -2.2222,  ..., -2.2222, -2.2222, -2.2222],
         [-2.2222, -2.2222, -2.2222,  ..., -2.2222, -2.2222, -2.2222],
         [-2.2222, -2.2222, -2.2222,  ..., -2.2222, -2.2222, -2.2222],
         ...,
         [ 0.5142,  0.6536,  0.5316,  ...,  0.7756,  0.6362,  0.1830],
         [ 0.5664,  0.6013,  0.2004,  ...,  0.4793,  0.3573,  0.2702],
         [-0.1656, -0.0959, -0.3399,  ..., -0.0261, -0.1307, -0.4444]]]), (2, 8, 0))


In [6]:
filename = 'AlexNetMulti--{}_augmentation-{}.md'.format(train_ds.name, str(train_ds.augmentation).lower())
best_model_file_path = os.path.join(RESULTS_FOLDER, filename)
callback_train_acc = callbacks.EpochScoring(scoring="accuracy", 
                                            lower_is_better=True, 
                                            on_train=True, 
                                            name='train_acc')
callback_save_best = callbacks.Checkpoint(monitor='valid_acc_best', 
                                          f_params=None, 
                                          f_optimizer=None, 
                                          f_criterion=None, 
                                          f_history=None, 
                                          f_pickle=best_model_file_path,  
                                          event_name='event_cp')
callback_early_stop = callbacks.EarlyStopping(monitor='valid_loss', patience=15, 
                                              threshold_mode='rel', lower_is_better=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [7]:
model = md_multi.AlexNetMultiTask()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [10]:
class NeuralNetMultiClassifier(NeuralNetClassifier):
    def get_loss(self, y_pred, y_true, *args, **kwargs):
        emotion_pred, speaker_pred, gender_pred = y_pred
        emotion_true, speaker_true, gender_true = y_true
        emotion_loss = super().get_loss(emotion_pred, emotion_true, *args, **kwargs)
        speaker_loss = super().get_loss(speaker_pred, speaker_true, *args, **kwargs)
        gender_loss = super().get_loss(gender_pred, gender_true, *args, **kwargs)
        loss = emotion_loss + speaker_loss + gender_loss
        return loss

In [12]:
net = NeuralNetMultiClassifier(
    model, criterion=nn.CrossEntropyLoss, optimizer=torch.optim.Adam,
    lr=0.001, max_epochs=100, batch_size=64, 
#     train_split=predefined_split(test_ds), 
    device=device, iterator_train__shuffle=True, 
    callbacks=[
        callback_train_acc,
        callback_save_best,
        callback_early_stop
    ]
)

In [17]:
net.fit(test_ds, y=None)

Re-initializing optimizer.


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 64 and the array at index 13 has size 61

In [15]:
history = net.history

In [16]:
print(history)

[{'batches': [{'train_loss': 4.386849403381348, 'train_batch_size': 64}, {'train_loss': 8.269963264465332, 'train_batch_size': 64}, {'train_loss': 4.390438556671143, 'train_batch_size': 64}, {'train_loss': 4.388858318328857, 'train_batch_size': 64}, {'train_loss': 4.388477325439453, 'train_batch_size': 64}, {'train_loss': 4.370421886444092, 'train_batch_size': 64}, {'train_loss': 4.360799312591553, 'train_batch_size': 64}, {'train_loss': 4.528645038604736, 'train_batch_size': 64}, {'train_loss': 4.36349630355835, 'train_batch_size': 64}, {'train_loss': 4.360868453979492, 'train_batch_size': 64}, {'train_loss': 4.361743927001953, 'train_batch_size': 64}, {'train_loss': 4.349571228027344, 'train_batch_size': 64}, {'train_loss': 4.3604254722595215, 'train_batch_size': 64}, {'train_loss': 4.372594833374023, 'train_batch_size': 64}, {'train_loss': 4.303104400634766, 'train_batch_size': 64}, {'train_loss': 4.354106903076172, 'train_batch_size': 64}, {'train_loss': 4.310717582702637, 'train_b